In [4]:
import os
import PIL
import json
import requests
from artist_name_format import correct_artist_name

working_path = '/Users/tod/todgru/notes/pdev/wikiart-working'

classes_to_oversample = [
    'Abstract Art',
    'Abstract Expressionism', # grouped with 'Action painting'
    'Naïve Art (Primitivism)',
    'Op Art',
    'Suprematism', # grouped with 'Neo-Suprematism',
    'Street art',    
]
classes = [
    'Surrealism',
    'Expressionism',
    'Cubism',
    'Pop Art',
    *classes_to_oversample
]

print(classes)
print("number of classes:", len(classes))


['Surrealism', 'Expressionism', 'Cubism', 'Pop Art', 'Abstract Art', 'Abstract Expressionism', 'Naïve Art (Primitivism)', 'Op Art', 'Suprematism', 'Street art']
number of classes: 10


In [10]:
path_to_wikiart = '/Users/tod/todgru/notes/pdev/wikiart/wikiart-saved'

# images sorted by artist and year
PATH_IMAGES = os.path.join(path_to_wikiart, "images")

# meta data json files about artists individual paintings
PATH_META = os.path.join(path_to_wikiart, "meta")

exampleArtists = {}
for c in classes:
    exampleArtists[c] = []

def parse_style_field(row):
    if row == None:
        return
    style_list = row.split(",")

    style = style_list[0]
    
    if style not in classes:
        return
    
    if style == 'Neo-Suprematism':
        return 'Suprematism'
    if style == 'Action painting':
        return 'Abstract Expressionism'
    return style

def check_if_image_is_valid(url, check_url=False):
    url = url.split("!")[0]
    return url
    # do not check url unless generating new data
    if(check_url):
        response = requests.head(url)
        print(str(response.status_code) + " : " + url)
        if(response.status_code == 200):
            return url
        return

for artist_meta_filename in os.listdir(PATH_META):
    #
    # don't include these files when collating file name and art style
    if artist_meta_filename not in [ 'artists.json', 'original-artists.json']:
        
        # read file
        with open(os.path.join(PATH_META, artist_meta_filename), 'r') as myfile:
            data=myfile.read()
        obj = json.loads(data)
        
        for artwork in obj:                
            temp = {}
            temp["artistName"] = correct_artist_name[artwork['artistName'].rstrip()]
            temp["title"] = artwork['title']
            temp["style"] = parse_style_field(artwork['style'])
            temp["link"] = "https://www.wikiart.org/en/{}".format(artwork['artistUrl'])
            temp["imageUrl"] = check_if_image_is_valid(artwork['image'])

            # art meta: artistName, title, image, style, link:https://www.wikiart.org/en/artistUrl  
            if temp["imageUrl"] == None:
                continue
            if temp["style"] != None:
                s = temp["style"]
                exampleArtists[temp["style"]].append(temp)
                

s = 0
for k in exampleArtists:
    t = len(exampleArtists[k])
    s = s+t
    print ("example paintings per style, {}:".format(k), t)
print("total example paintings", s)

# Write json to file
with open('/Users/tod/todgru/notes/pdev/art-classifier-ml-deploy/exampleArtists.json', 'w', encoding='utf-8') as f:
    json.dump(exampleArtists, f, ensure_ascii=False, indent=4)
            
            

example paintings per style, Surrealism: 784
example paintings per style, Expressionism: 562
example paintings per style, Cubism: 411
example paintings per style, Pop Art: 341
example paintings per style, Abstract Art: 210
example paintings per style, Abstract Expressionism: 41
example paintings per style, Naïve Art (Primitivism): 192
example paintings per style, Op Art: 165
example paintings per style, Suprematism: 118
example paintings per style, Street art: 77
total example paintings 2901
